# IPT

In [1]:
using PyPlot

rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")

rcParams["text.usetex"] = true
rcParams["font.family"] = "serif"
rcParams["font.size"] = 16
rcParams["text.latex.preamble"] = raw"\usepackage{amsmath}"

"\\usepackage{amsmath}"

In [2]:
import PyCall: pyimport
irbasis3 = pyimport("irbasis3")

PyObject <module 'irbasis3' from '/home/shinaoka/.local/lib/python3.8/site-packages/irbasis3/__init__.py'>

## Step 1: Generate IR basis and associated sampling points

In [3]:
beta = 10.0
wmax = 10.0
lambda_ = beta * wmax
eps = 1e-7

wmax = lambda_/beta
K = irbasis3.KernelFFlat(lambda_=lambda_)
basis = irbasis3.FiniteTempBasis(K, statistics="F", beta=beta, eps=eps)
L = basis.size

# Sparse sampling in tau
smpl_tau = irbasis3.TauSampling(basis)
ntau = length(smpl_tau.sampling_points)
print("cond (tau): ", smpl_tau.cond)

# Sparse sampling in Matsubara frequencies
smpl_matsu = irbasis3.MatsubaraSampling(basis)
nw = length(smpl_matsu.sampling_points)
print("cond (matsu): ", smpl_matsu.cond)

cond (tau): 5.818091567533522

cond (matsu): 9.633471018104835

## Step 2
Define a non-interacting DOS:
$$
\rho(\omega) = \frac{\sqrt{4-\omega^2}}{2\pi}.
$$

In [4]:
using QuadGK

rho_omega(omega) = sqrt.(4-omega^2) / (2 * pi)

@assert isapprox(quadgk(rho_omega, -2, 2)[1], 1, atol=1e-9)

## Self-consistent equation

$$
\begin{align*}
  G_\mathrm{loc}(\mathrm{i}\nu) &= \int \mathrm{d}\omega \frac{\rho(\omega)}{\mathrm{i}\nu - \omega - \Sigma(\mathrm{i}\nu)}\\
  \mathcal{G}(\mathrm{i}\nu) &= (G^{-1}(\mathrm{i}\nu) + \Sigma(\mathrm{i}\nu))^{-1}\\
  \Sigma(\tau) &= U^2 \mathcal{G}(\tau)^2 \mathcal{G}(\beta-\tau)
\end{align*}
$$

In [5]:
"""
Scale weights and notes of quadrature to the interval [xmin, xmax]
"""
using FastGaussQuadrature

function scale_quad(x::Vector{Float64}, w::Vector{Float64}, xmin::Flot64, xmax::Float64)
    @assert xmin < xmax
    dx = xmax - xmin
    w_ = 0.5 * dx * w
    x_ = (0.5 * dx) * (x + 1) + xmin
    return x_, w_
end

struct IPTSolver
    U::Float64
    basis
    beta::Float64
    rho_omega::Function
    quad_weight::Vector{Float64}
    quad_omega::Vector{Float64}
    smpl_matsu
    smpl_tau
end

function IPTSolver(U::Float64, basis, rho_omega::Function,
    omega_min::Float64, omega_max::Float64, deg_leggauss::Int64=1000)
    @assert omega_min < omega_max

    x_, w_ = gausslegendre(deg_leggauss)
    quad_omega, quad_weight = scale_quad(x_, w_, omega_min, omega_max)
    smpl_matsu = irbasis3.MatsubaraSampling(basis)
    smpl_tau = irbasis3.TauSampling(basis)
    IPTSolver(
        U,
        basis,
        rho_omega,
        quad_weight,
        quad_omega,
        smpl_matsu,
        smpl_tau
    )
end

function gloc_iv(solver::IPTSolver, sigma_iv::Vector{Complex{Float64}})
    """
    Compute G_loc(iv) from Sigma_iv

    G_loc(iv) \simeq \sum_q w_q rho(x_q)/(iv - w_q - \Sigma(iv))
    """
    iv = 1im * solver.smpl_matsu.sampling_points * π/solver.beta
    gloc_iv = Vector{Complex{64}}(undef, length(iv))
    for iq in 1:length(quad_weight)
    end
    r = np.einsum('q,q,wq->w',
            quad_weight,
            self.rho_omega(solver.quad_weight),
            1/((iv-sigma_iv)[:,None] - solver.quad_omega[None,:]),
            optimize=True
        )
    return r
end
    
    def g0_iv(self, gloc, sigma_iw):
        """
        Compute \mathcal{G}(iv) from G_loc(iv) and Sigma(iv)
        """
        return 1/(1/gloc + sigma_iw)
    
    def sigma_iv(self, g0_iv):
        """
        Compute Sigma(tau) from \mathcal{G}(iv)
        """
        g0_IR = self._smpl_matsu.fit(g0_iv, axis=0)
        g0_tau = self._smpl_tau.evaluate(g0_IR)
        sigma_tau = (self.U**2) * (g0_tau**2) * g0_tau[::-1]
        sigma_IR = self._smpl_tau.fit(sigma_tau)
        return self._smpl_matsu.evaluate(sigma_IR, axis=0)
    
    def new_sigma_iv(self, sigma_iv):
        gloc_iv = self.gloc_iv(sigma_iv)
        g0_iv = self.g0_iv(gloc_iv, sigma_iv)
        r = self.sigma_iv(g0_iv)
        return r


ErrorException: cannot document the following expression:

using FastGaussQuadrature


In [6]:
niter = 1000
mixing = 0.1

U = 1.0

# Initialize solver
solver = IPTSolver(U, basis, rho_omega, (-2,2))

sigma_iv = np.zeros_like(
    solver._smpl_matsu.sampling_points.size, dtype=np.complex128)


UndefVarError: UndefVarError: IPTSolver not defined

In [7]:
# Gloc_iv
gloc_iv = solver.gloc_iv(sigma_iv)
plt.plot(
    solver._smpl_matsu.sampling_points,
    gloc_iv.real
)
plt.plot(
    solver._smpl_matsu.sampling_points,
    gloc_iv.imag
)

UndefVarError: UndefVarError: solver not defined

In [8]:
# Gloc_iv
g0_iv = solver.g0_iv(gloc_iv, sigma_iv)
plt.plot(
    solver._smpl_matsu.sampling_points,
    g0_iv.real
)
plt.plot(
    solver._smpl_matsu.sampling_points,
    g0_iv.imag
)

UndefVarError: UndefVarError: solver not defined

In [9]:
for iter in range(niter):
    new_sigma_iv = solver.new_sigma_iv(sigma_iv)
    rdiff = np.abs(new_sigma_iv - sigma_iv).max()/np.abs(sigma_iv).max()
    print(iter, rdiff)
    if rdiff < 1e-10:
        break
    sigma_iv = mixing * new_sigma_iv + (1-mixing) * sigma_iv

ErrorException: syntax: line break in ":" expression

In [10]:
plt.plot(
    solver._smpl_matsu.sampling_points,
    sigma_iv.imag,
    marker="x"
)

UndefVarError: UndefVarError: solver not defined